# Preprocessing Event Data

In [5]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium

In [6]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("ADS Project1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

In [7]:
spark.conf.set("spark.sql.parquet.compression.codec","gzip")

In [9]:
# Path to your CSV file
csv_file_path = "../data/landing/external/NYC_Permitted_Event_Information_-_Historical.csv"

# Read the CSV file into a DataFrame
edf = spark.read.csv(csv_file_path, header=True, inferSchema=True)

In [10]:
# Show the DataFrame schema
edf.printSchema()

root
 |-- Event ID: integer (nullable = true)
 |-- Event Name: string (nullable = true)
 |-- Start Date/Time: string (nullable = true)
 |-- End Date/Time: string (nullable = true)
 |-- Event Agency: string (nullable = true)
 |-- Event Type: string (nullable = true)
 |-- Event Borough: string (nullable = true)
 |-- Event Location: string (nullable = true)
 |-- Event Street Side: string (nullable = true)
 |-- Street Closure Type: string (nullable = true)
 |-- Community Board: string (nullable = true)
 |-- Police Precinct: string (nullable = true)



In [11]:
# Show the first 5 rows
edf.show(5)

+--------+--------------------+--------------------+--------------------+--------------------+--------------+-------------+--------------------+-----------------+--------------------+---------------+---------------+
|Event ID|          Event Name|     Start Date/Time|       End Date/Time|        Event Agency|    Event Type|Event Borough|      Event Location|Event Street Side| Street Closure Type|Community Board|Police Precinct|
+--------+--------------------+--------------------+--------------------+--------------------+--------------+-------------+--------------------+-----------------+--------------------+---------------+---------------+
|  368421|    Big Apple Circus|11/18/2017 07:00:...|11/18/2017 08:00:...|    Parks Department| Special Event|    Manhattan|Damrosch Park: Da...|             NULL|                 N/A|            7, |           20, |
|  330050|Mt. Eden Farmer's...|11/16/2017 08:00:...|11/16/2017 04:00:...|    Parks Department| Special Event|        Bronx|Mount Eden Ma

In [12]:
# Select the columns: Event ID, Event Type, Start Date/Time, End Date/Time, Event Borough, Event Location
edf = edf.select("Event ID", "Event Type", "Start Date/Time", "End Date/Time", "Event Borough", "Event Location")
edf.show(5)


+--------+--------------+--------------------+--------------------+-------------+--------------------+
|Event ID|    Event Type|     Start Date/Time|       End Date/Time|Event Borough|      Event Location|
+--------+--------------+--------------------+--------------------+-------------+--------------------+
|  368421| Special Event|11/18/2017 07:00:...|11/18/2017 08:00:...|    Manhattan|Damrosch Park: Da...|
|  330050| Special Event|11/16/2017 08:00:...|11/16/2017 04:00:...|        Bronx|Mount Eden Malls:...|
|  314111|Farmers Market|11/21/2017 08:00:...|11/21/2017 05:00:...|    Manhattan| BROADWAY between...|
|  369850|  Construction|11/23/2017 12:00:...|11/23/2017 11:58:...|    Manhattan|Madison Square Pa...|
|  335783| Special Event|11/22/2017 09:00:...|11/22/2017 08:00:...|Staten Island|LaTourette Park &...|
+--------+--------------+--------------------+--------------------+-------------+--------------------+
only showing top 5 rows



In [13]:
from pyspark.sql.functions import col, to_timestamp, lit

# Convert the 'Start Date/Time' and 'End Date/Time' columns to timestamp type using the correct format
edf = edf.withColumn("Start Date/Time", to_timestamp(col("Start Date/Time"), "MM/dd/yyyy hh:mm:ss a")) \
       .withColumn("End Date/Time", to_timestamp(col("End Date/Time"), "MM/dd/yyyy hh:mm:ss a"))

# Define the date range as timestamps in the correct format
start_date = "2023-12-01 00:00:00"
end_date = "2024-05-31 23:59:59"

# Convert start_date and end_date to timestamp using lit and to_timestamp
start_timestamp = to_timestamp(lit(start_date), "yyyy-MM-dd HH:mm:ss")
end_timestamp = to_timestamp(lit(end_date), "yyyy-MM-dd HH:mm:ss")

# Filter the DataFrame to include only rows within the specified date range
edf_filtered = edf.filter((col("Start Date/Time") >= start_timestamp) & 
                          (col("End Date/Time") <= end_timestamp))

# Show the filtered DataFrame
edf_filtered.show(5) 

+--------+-------------+-------------------+-------------------+-------------+--------------------+
|Event ID|   Event Type|    Start Date/Time|      End Date/Time|Event Borough|      Event Location|
+--------+-------------+-------------------+-------------------+-------------+--------------------+
|  684438|Special Event|2023-12-09 07:00:00|2023-12-09 10:00:00|     Brooklyn|Prospect Park: Pi...|
|  684437|Special Event|2023-12-02 07:00:00|2023-12-02 10:00:00|     Brooklyn|Prospect Park: Pi...|
|  684438|Special Event|2023-12-09 07:00:00|2023-12-09 10:00:00|     Brooklyn|Prospect Park: Pi...|
|  684437|Special Event|2023-12-02 07:00:00|2023-12-02 10:00:00|     Brooklyn|Prospect Park: Pi...|
|  684438|Special Event|2023-12-09 07:00:00|2023-12-09 10:00:00|     Brooklyn|Prospect Park: Pi...|
+--------+-------------+-------------------+-------------------+-------------+--------------------+
only showing top 5 rows



## Missing values 

In [14]:
# Check missing values 
edf_filtered.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in edf_filtered.columns]).show()

+--------+----------+---------------+-------------+-------------+--------------+
|Event ID|Event Type|Start Date/Time|End Date/Time|Event Borough|Event Location|
+--------+----------+---------------+-------------+-------------+--------------+
|       0|         0|              0|            0|            0|             0|
+--------+----------+---------------+-------------+-------------+--------------+



No Missing entries

## Export to raw folder

In [15]:
# Export the filtered DataFrame to a Parquet file in the raw zone
edf_filtered.write.mode("overwrite").parquet("../data/raw/NYC_Permitted_Event_Information_Historical.parquet")

24/08/26 22:11:45 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
24/08/26 22:11:46 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
24/08/26 22:11:46 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
24/08/26 22:11:46 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
24/08/26 22:11:46 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
